In [102]:
import sys
import os
import tqdm
import collections
import re
import pickle

import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from transformers import ElectraModel, ElectraTokenizer
from transformers import RobertaModel, RobertaTokenizer

In [143]:
# model = BertModel.from_pretrained('hfl/chinese-bert-wwm-ext')
# tokenizer = BertTokenizer.from_pretrained('hfl/chinese-bert-wwm-ext')
# model = ElectraModel.from_pretrained('hfl/chinese-electra-180g-small-discriminator')
# tokenizer = ElectraTokenizer.from_pretrained('hfl/chinese-electra-180g-small-discriminator')
model = BertModel.from_pretrained('hfl/chinese-roberta-wwm-ext')
tokenizer = BertTokenizer.from_pretrained('hfl/chinese-roberta-wwm-ext')

Downloading:   0%|          | 0.00/412M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [65]:
DATA_PATH = '../data/kindle-完善数据.xlsx'
PUNC_REGEX = "！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏."

In [14]:
df = pd.read_excel(DATA_PATH)
df

,品牌或店铺名称,问题类别,标准问题,标准问题的其他问法,标准答案,Unnamed: 5,Unnamed: 6
0,kindle官方旗舰店,物流相关问题|配送,商品包邮吗？,是否包邮？\n免费配送吗？\n是否免费配送？\n有免费配送吗？\n含包邮服务吗？,kindle官方旗舰店为大陆地区顾客提供全场包邮服务。快速高效，免费配送、请亲放心购买哦！,NaN,NaN
1,kindle官方旗舰店,物流相关问题|配送,发什么快递,发什么快递\n从哪里发货\n可以指定快递吗\n发什么快递\n物流是什么,我们用淘宝旗下的菜鸟物流，菜鸟物流是由圆通，中通，韵达，EMS等快递配送的，系统会根据地址自...,NaN,NaN
2,kindle官方旗舰店,物流相关问题|配送,什么时候发货,今天能发货吗\n现在买什么时候发货\n什么时候发货\n发货时间,正常情况下当天下午四点下单，预计当天可以发货；当天下午四点后下单预计第二天发货，可以参照页面...,NaN,NaN
3,kindle官方旗舰店,产品相关问题|打招呼,你好，在吗,在吗\n你好\n请问\nhello,对商品活动有疑问可以留下您的问题哦,NaN,NaN
4,kindle官方旗舰店,产品相关问题|配置信息,是官方正品吗？,是正品吗\n是全新机器吗\n是国行吗\n是什么版本的\n不会是翻新的吧\n是美版吗\n产自哪...,我们是kindle在天猫自营的唯一官方旗舰店，所售商品均为全新正品国行，享受亚马逊官方质保呢...,NaN,NaN
...,...,...,...,...,...,...,...
128,kindle官方旗舰店,通用话术|打招呼,你好,你好\n在吗？\n在嘛？\n您好\nHi\nHello ...,你好啊，欢迎来到直播间，有什么问题都可以直接问我哦。,NaN,NaN
129,kindle官方旗舰店,通用话术|打招呼,能问问题吗？,能回答问题吗？\n有问题能问你吗？\n能提问吗？\n你能回答我的问题吗？\n怎么没有互动啊,我可以互动的，宝宝们随时提问哦，介绍完商品我就会给大家回答的哦，小吉还在学习中，我会越来越聪明的。,NaN,NaN
130,kindle官方旗舰店,通用话术|打招呼,是直播吗？,这是啥？\n这是直播吗？\n是实时互动的吗？\n 你能回复人吗,这里是直播哦，我是智能主播小吉，大家可以发消息和我聊聊天哦。,NaN,NaN
131,kindle官方旗舰店,通用话术|打招呼,是真人配音吗？,声音是真人配音吗？\n主播声音有点沙哑 \n主播声音真好听,小吉的声音是不是有些特别啊，因为小吉用的是人工智能合成的声音哦。,NaN,NaN


In [156]:
def remove_punc(lines):
    if type(lines) == str:
        ret = re.sub(r"[%s]+" %PUNC_REGEX, '', lines)
        return ret
    
    ret = []
    for line in lines:
        temp = re.sub(r"[%s]+" %PUNC_REGEX, '', line)
        ret.append(temp)
    return ret
        
def search_query(query, all_questions, question_vectors):
    query = remove_punc(query).strip()
    inputs = tokenizer(query, return_tensors = 'pt')
    outputs = model(**inputs)
    vector = outputs.last_hidden_state[0][0].detach().numpy()
    distance = {}
    for ind, ques_vector in enumerate(question_vectors):
        cur_distance = np.linalg.norm(ques_vector - vector)
        distance[round(cur_distance,2)] = all_questions[ind]
    return distance

In [132]:
# query = '这个标准问题'
# inputs = tokenizer(query, return_tensors = 'pt')
# print(inputs)
# outputs = model(**inputs)
# vector = outputs.last_hidden_state
# print(vector[0][0])

In [158]:
temp = df['标准问题的其他问法'].tolist()
extend_questions = []
for ques in temp:
    extend_questions.extend(ques.split('\n'))
# print(len(extend_questions))
standard_questions = df['标准问题'].tolist()
all_questions = extend_questions + standard_questions
all_questions = [x.strip() for x in all_questions]
all_questions = [x for x in all_questions if x]
all_questions = remove_punc(all_questions)
print(len(all_questions))

583


In [159]:
question_vectors = []
for ques in all_questions:
    inputs = tokenizer(ques, return_tensors = 'pt')
#     print(inputs)
#     sent_ids = inputs['input_ids']
#     attn_mask = inputs['attention_mask']
#     outputs = model(sent_ids, attn_mask)
    outputs = model(**inputs)
#     print(outputs)
    vector = outputs.last_hidden_state[0][0]
#     print(len(vector[0][0]))
    question_vectors.append(vector.detach().numpy())

In [160]:
len(question_vectors[0])

768

In [166]:
query = '怎么？不能联网呢'
out_dict  = search_query(remove_punc(query))
ordered_dict = collections.OrderedDict(sorted(out_dict.items(), key = lambda x:x[0], reverse = False))
ordered_dict

OrderedDict([(6.27, '不能联网'),
             (7.07, '能联网吗'),
             (9.02, '怎么不理人'),
             (9.84, '怎么没有互动啊'),
             (10.07, '能提问吗'),
             (10.21, '能问问题吗'),
             (10.35, '不联网可以看吗'),
             (10.47, '有问题能问你吗'),
             (10.5, '怎么不回答问题'),
             (10.62, '是实时互动的吗'),
             (10.64, '都有什么功能啊'),
             (10.69, '你能回答我的问题吗'),
             (10.75, '不理人'),
             (10.81, '怎么这么耗电'),
             (11.03, '请问'),
             (11.04, '是直播吗'),
             (11.17, '有笔吗'),
             (11.2, '我看评论有问题的话联系维修什么的不是很效率'),
             (11.21, '你能回复人吗'),
             (11.3, 'kindle可以使用手机热点联网吗'),
             (11.31, '怎么下书'),
             (11.33, '可以看小说吗'),
             (11.34, '耗电怎么这么快'),
             (11.37, '你是谁'),
             (11.38, '有广告吗'),
             (11.4, '能插电话卡'),
             (11.43, '我以后怎么保修呢'),
             (11.44, '有什么优惠呢'),
             (11.47, '8g会不会卡'),
             (11.48, '会员什么价啊'),
             (11.5, '直播间不是需要备注的吗'),
  

In [5]:
a = [
    {123:123},
    {123:456}
]
sorted(a, key = lambda x:x[123])

[{123: 123}, {123: 456}]

In [50]:
import sys
import os
import collections
import re

from tqdm import tqdm
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel


model = BertModel.from_pretrained('hfl/chinese-roberta-wwm-ext')
tokenizer = BertTokenizer.from_pretrained('hfl/chinese-roberta-wwm-ext')


def remove_punc(lines):
    PUNC_REGEX = "！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏."
    if type(lines) == str:
        ret = re.sub(r"[%s]+" %PUNC_REGEX, '', lines)
        return ret
    
    ret = []
    for line in lines:
        temp = re.sub(r"[%s]+" %PUNC_REGEX, '', line)
        ret.append(temp)
    return ret

def read_ques_anws_from(filename):
    df = pd.read_excel(filename)
    ret = []
    for ind in df.index:
        cur_anw = df.loc[ind, '标准答案'].strip()
        cur_std_ques = df.loc[ind, '标准问题']
        cur_add_ques = df.loc[ind, '标准问题的其他问法'].split('\n')
        cur_all_ques = [cur_std_ques] + cur_add_ques
        cur_all_raw_ques = [x.strip() for x in cur_all_ques]
        cur_all_ques = [remove_punc(x) for x in cur_all_raw_ques if x]
        for ind, cur_ques in enumerate(cur_all_ques):
            cur_dict = {}
            cur_dict['raw_question'] = cur_all_raw_ques[ind]
            cur_dict['question'] = cur_ques
            cur_dict['answer'] = cur_anw
            ret.append(cur_dict)
    return ret

def dict_add_vectors(dic_list, model, tokenizer):
    cur_dic_list = dic_list.copy()
    for dic in tqdm(cur_dic_list):
        text = dic['question']
        inputs = tokenizer(text, return_tensors = 'pt')
        outputs = model(**inputs)
        vector = outputs.last_hidden_state[0][0]
        dic['vector'] = vector.detach().numpy()
    return dic_list

def search_query(query, dic_list, model, tokenizer):
    cur_dic_list = dic_list.copy()
    query = remove_punc(query).strip()
    inputs = tokenizer(query, return_tensors = 'pt')
    outputs = model(**inputs)
    query_vec = outputs.last_hidden_state[0][0].detach().numpy()
    for dic in tqdm(cur_dic_list):
        dic_vec = dic['vector']
        cur_distance = np.linalg.norm(dic_vec - query_vec)
        dic['distance'] = cur_distance
    cur_dic_list = sorted(dic_list, key = lambda x: x['distance'])
    ret = []
    for dic in cur_dic_list:
        cur_dic = {}
        cur_dic['raw_question'] = dic['raw_question']
        cur_dic['question'] = dic['question']
        cur_dic['distance'] = dic['distance']
        cur_dic['answer'] = dic['answer']
        ret.append(cur_dic)
    return ret[:5]

In [51]:
dic_list = read_ques_anws_from('../data/kindle-完善数据.xlsx')
dic_list_with_vec = dict_add_vectors(dic_list, model, tokenizer)
# query = '能看图片嘛？'
# output = search_query(query, dic_list, model, tokenizer)
# print(output)

100%|██████████| 583/583 [00:20<00:00, 28.10it/s]


In [53]:
query = '包邮'
output = search_query(query, dic_list_with_vec, model, tokenizer)
print(output)

100%|██████████| 583/583 [00:00<00:00, 116747.64it/s]

[{'raw_question': '是否包邮？', 'question': '是否包邮', 'distance': 6.960829, 'answer': 'kindle官方旗舰店为大陆地区顾客提供全场包邮服务。快速高效，免费配送、请亲放心购买哦！'}, {'raw_question': '商品包邮吗？', 'question': '商品包邮吗', 'distance': 8.739631, 'answer': 'kindle官方旗舰店为大陆地区顾客提供全场包邮服务。快速高效，免费配送、请亲放心购买哦！'}, {'raw_question': '含包邮服务吗？', 'question': '含包邮服务吗', 'distance': 9.754724, 'answer': 'kindle官方旗舰店为大陆地区顾客提供全场包邮服务。快速高效，免费配送、请亲放心购买哦！'}, {'raw_question': 'hello', 'question': 'hello', 'distance': 10.768649, 'answer': '对商品活动有疑问可以留下您的问题哦'}, {'raw_question': 'Hello', 'question': 'Hello', 'distance': 10.768649, 'answer': '你好啊，欢迎来到直播间，有什么问题都可以直接问我哦。'}]


In [85]:
temp = str(dic_list_with_vec[0]['vector'])[1:-1].split(' ')
# temp = temp[1:-1]
temp = [x.strip('\n') for x in temp]
temp = [float(x) for x in temp if x]
temp
# 0.311205328

[0.311205328,
 -0.0712500289,
 0.284049779,
 0.236432016,
 -0.640770793,
 0.442236274,
 -0.491577834,
 1.07718432,
 0.0675739795,
 0.56934458,
 0.647478998,
 1.28442466,
 1.21415353,
 -0.850115001,
 -1.11348999,
 0.616038501,
 0.392589837,
 0.963288307,
 -0.822674394,
 -1.26043844,
 1.00226247,
 -0.166257426,
 -1.66553044,
 0.61760658,
 -0.748430252,
 -0.0186609998,
 -0.496871412,
 0.189372852,
 -0.937361956,
 0.154007465,
 0.260031283,
 -0.231249169,
 -1.03519142,
 0.0892058536,
 -0.485512167,
 -0.165308908,
 0.772662878,
 -1.20428145,
 -0.367789596,
 -0.843663335,
 -0.316298276,
 0.387321621,
 -1.97115898,
 0.646403849,
 -0.383771449,
 0.573863328,
 0.163997501,
 0.576312244,
 -1.2897613,
 0.925834954,
 0.628125787,
 6.14975786,
 0.42242375,
 0.690511048,
 0.507393718,
 1.29224455,
 -0.333694279,
 -0.245621651,
 0.490015894,
 -0.389181346,
 -0.165709093,
 0.0612660833,
 0.96917522,
 0.187116668,
 0.357750833,
 -0.163960874,
 -0.561261058,
 0.0331548899,
 -0.786950827,
 -0.460401177,


In [86]:
a = 'insert into filter_question(name, raw_question, \
question, answer, answer_set, vector) values (%s, %s, %s, %s, %s, %s)'

In [91]:
test = dic_list_with_vec[0]['vector']

In [93]:
data = test.dumps()

In [106]:
len(data)

4846

In [103]:
pickle.loads(data)

array([ 3.11205328e-01, -7.12500289e-02,  2.84049779e-01,  2.36432016e-01,
       -6.40770793e-01,  4.42236274e-01, -4.91577834e-01,  1.07718432e+00,
        6.75739795e-02,  5.69344580e-01,  6.47478998e-01,  1.28442466e+00,
        1.21415353e+00, -8.50115001e-01, -1.11348999e+00,  6.16038501e-01,
        3.92589837e-01,  9.63288307e-01, -8.22674394e-01, -1.26043844e+00,
        1.00226247e+00, -1.66257426e-01, -1.66553044e+00,  6.17606580e-01,
       -7.48430252e-01, -1.86609998e-02, -4.96871412e-01,  1.89372852e-01,
       -9.37361956e-01,  1.54007465e-01,  2.60031283e-01, -2.31249169e-01,
       -1.03519142e+00,  8.92058536e-02, -4.85512167e-01, -1.65308908e-01,
        7.72662878e-01, -1.20428145e+00, -3.67789596e-01, -8.43663335e-01,
       -3.16298276e-01,  3.87321621e-01, -1.97115898e+00,  6.46403849e-01,
       -3.83771449e-01,  5.73863328e-01,  1.63997501e-01,  5.76312244e-01,
       -1.28976130e+00,  9.25834954e-01,  6.28125787e-01,  6.14975786e+00,
        4.22423750e-01,  

In [108]:
a = 0
assert a

AssertionError: 

In [109]:
348 * 768

267264

In [114]:
data = {}
key = 'foo'
value = []
data.setdefault(key, value)
print(data)
value.append('hello')
data

{'foo': []}


{'foo': ['hello']}

In [121]:
a = {}
names = a.setdefault('abc', [])
names.append('sdadasda')
a

{'abc': ['sdadasda']}

In [130]:
visits = {
    'Mexico':{'Tulum', 'Puerto Vallarta'},
    'Japan':{'Hakone'},
}

In [131]:
visits.setdefault('Japan', set())

{'Hakone'}

In [128]:
visits

{'Mexico': {'Puerto Vallarta', 'Tulum'}, 'Japan': {'Hakone'}, 'France': set()}

In [140]:
a = np.array([1,2,3,4,5])
a

array([1, 2, 3, 4, 5])

In [141]:
a = np.delete(a, [1,2], axis = 0)

In [142]:
a

array([1, 4, 5])

In [79]:
from datetime import datetime

In [127]:
a = {}

In [128]:
now = datetime.now()
for i in range(10**7):
    a.setdefault('kindle', []).append([222])
#     if not 'kindle' in a:
#         a['kindle'] = []
#     a['kindle'].append(222)
#     print(a)
print(datetime.now() - now)

0:00:05.007128


In [4]:
x

[123]

In [1]:
a = []
a.append([])

In [9]:
import numpy as np

In [152]:
a = np.array([])
b = np.array([[5,6,7]])
a

array([], dtype=float64)

In [153]:

# x.append(a)
# x.append(b)
y = np.append(a,b, axis = 0)
y

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)

In [71]:
x[0] == y[0]

True

In [45]:
x = np.array(x)
x

array([[1, 2, 3],
       [4, 5, 6]])

In [136]:
vectors = np.load('../np_array/4109_18972957475_kindle.pkl', allow_pickle= True)
vectors

array([], dtype=float64)

In [143]:
a = np.array([])
b = np.array([[1,2]])
c = np.append(a,b, axis = 0)
c

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)